In [1]:
Zx.<x> = ZZ[]
def Convolution_in_R(f,g,p):
    return (f*g)%(x^p-x-1)

def matrix_for_the_lattice(p,q,F):
    H = matrix(p)
    I=identity_matrix(p)

    Zero_Matrix=matrix(p)

    for i in range(p):
        for j in range(p):
            H[i,j] = Convolution_in_R(Zx(F),x^i,p)[j]
       
    B_1=block_matrix([[I,H]])       
    B_2=block_matrix([[Zero_Matrix,q*I]])
    M_NTRU=block_matrix([[B_1],[B_2]])
    return M_NTRU

def is_vfk(Q,N):
    '''
    Q is the Gram-Schimdt matrix of the superbasis
    N is the parameter given when we build ntru_vfk_matrix
    '''
    lst=[]
    x=np.array([Q])
    lst=x[np.nonzero(x)]
    lst=list(lst)
    for i in range(2*N+1):
        if Q[i][i]!=0:
            lst.remove(Q[i][i])
    return all(val <= 0 for val in lst)

def get_superbasis(basis): # basis is a list
    n = len(basis)
    basis = [vector(x) for x in basis]
    superbasis_vector = -sum(basis[i] for i in [0..n-1])
    return [vector(basis[i]) for i in [0..n-1]] + [vector(superbasis_vector)]

def get_qij(superbasis):
    n = len(superbasis)
    Q = [superbasis[i].dot_product(superbasis[j]) for i in range(n) for j in range(n)]
    Q = matrix(n, n, Q)
    return Q

def find_k_and_P(q):
    for k in range(1,200):
        A,B,C = -(1+k^2), -k^2+2*k*q-1, q*(k-q)
        g = A*x^2+B*x+C
        P = floor( k*q / (k^2+1)  )
        G=g.subs(x=P)
        if G>0:
            A,B,C = -(1+(k-1)^2), -(k-1)^2+2*(k-1)*q-1, q*(k-1-q)
            g = A*x^2+B*x+C
            P = floor( (k-1)*q / ( (k-1)^2+1)  )
            G=g.subs(x=P)
            print(q,":",k-1,P)  
            return k-1,P
            
            
def unimodular(P,N):
    '''
    N,P see the paper (both are positive integer)
    The output is a unimodular matrix of dimension 2N x 2N
    '''
    #upper block
    I=identity_matrix(N)
    Zero_Matrix=matrix(N)
    #lower block
    matrix_P = P*I
    
    B_u = block_matrix([[I,Zero_Matrix]])   
    B_l = block_matrix([[matrix_P,I]])  
    unimod_matrix = block_matrix([[B_u],[B_l]])   
    return unimod_matrix

def hits(L,M):
    Len = len(L)
    K = []
    for i in range(Len):
        if L[i]!=M[i]:
            K.append(i)
    Len1 = Integer(len(K))
    percentage = ((Len-Len1)/Len) * 100
    print("percentage:",float(percentage))
    print("the lists differs in ",Len1," elements")
    return 

In [3]:
import hashlib
import numpy as np
def hash(s): h = hashlib.sha512(); h.update(s); return h.digest()
def random32(): return randrange(-2^31,2^31)        # a random integer in -2^31 and 2^31
def random32even():  return random32() & (-2)       # takes the output of random32() and use the logical AND with (-2)
def random321mod4(): return (random32() & (-3)) | 1 # generate a random 32 bit integer that's congruent to 1 modulo 4

def randomrange3():  
    ''' 
    generate a random integer in {0, 1, 2} 
    by utilizing a 32-bit random number generator : random32()
    '''
    return ((random32() & 0x3fffffff) * 3) >> 30

def CenterLift(f,q,p):   
    f_balanced = list(   ((f[i]+q//2)%q) -q//2  for i in range(p))
    return Zx(f_balanced)

def nicelift(u):
    return lift(u + q//2) - q//2

def round_half_away_from_zero(x):
    return sgn(x)*floor(abs(x)+0.5)

def nicemod3(u): # r in {0,1,-1} with u-r in {...,-3,0,3,...}
    myround = round_half_away_from_zero
    return u - 3*myround(u/3)

def randomR(): # R element with 2t coeffs +-1
    L = [random32even() for i in range(2*t)]
    L += [random321mod4() for i in range(p-2*t)]
    L.sort()
    L = [(L[i]%4)-1 for i in range(p)]
    return Zx(L)

def lst_to_nicepol(L):
    # L is from LSB-->MSB
    L = [int(x) for x in L]
    p=len(L)
    f = sum(x^i*L[i] for i in range(p))
    return f

def unknown_u_vector(kappa,h,r):
    r = h * Rq(r) # this multiplication is in R/q. 
    u  = -kappa*hr # u : the unknwon vector u (u(x)=-k(h*r))
    u_list = Zx(u.lift()).list() # convert to list     
    u_vector = vector(u_list) #
    return u_vector

def from_Rq_back_to_R3(rq,p,q):
    temp_list = list(Rq(rq))[0:p]
    Z = []
    for z in temp_list:
        if int(z)>q/2:
            Z.append(int(z)-q)
        else:
            Z.append(z)
    return lst_to_nicepol(Z)

def find_integer_with_min_distance(a, b, c):
    min_distance = float('inf')  # Initialize with positive infinity.
    integer_with_min_distance = None

    # Iterate through integers starting from the minimum of a, b, and c to the maximum.
    for x in range(min(a, b, c), max(a, b, c) + 1):
        distance = max(abs(a - x) + abs(b - x) ,abs(c - x))
        #distance = (a - x)**2 + (b - x)**2 + (c - x)**2
        #distance = abs(a - x) + abs(b - x) + abs(c - x)
                
        # Check if the current distance is smaller than the minimum distance.
        if distance < min_distance:
            min_distance = distance
            integer_with_min_distance = x

    return integer_with_min_distance


def LLL_reduction_of_M_NTRU(init_M_NTRU):  
    from fpylll import GSO
    from fpylll import IntegerMatrix,LLL
    import time
    def fptosage(A):
        n = A.nrows
        C = matrix(n)
        for i in range(n):
            C[i] = list(A[i])
        return C  
    M_NTRU_fplll = IntegerMatrix.from_matrix(init_M_NTRU)
    LLL.reduction(M_NTRU_fplll, delta =0.99 )
    M_NTRU = fptosage(M_NTRU_fplll)
    print("LLL is done")
    return M_NTRU,M_NTRU_fplll

def correction_of_msg(N,m):
    m_list=m.list()
    if len(m_list)<N:
        diff = N - len(m_list)
        m_list.append(diff*[0])
    return flatten(m_list)